In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import re

In [2]:
data = pd.read_csv('archive/Sentiment.csv')

# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [3]:
def preProcess_data(text):
    text = text.lower()
    new_text = re.sub('[^a-zA-z0-9\s]','',text)
    new_text = re.sub('rt', '', new_text)
    return new_text

data['text'] = data['text'].apply(preProcess_data)

In [4]:
max_fatures = 2000

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, 28) 

Y = pd.get_dummies(data['sentiment']).values

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20)

In [6]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(128,recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])


In [7]:
batch_size = 512

model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, validation_data=(X_test, Y_test))


Epoch 1/10
22/22 [==============================] - 55s 2s/step - loss: 0.9581 - accuracy: 0.6014 - val_loss: 0.9113 - val_accuracy: 0.6105
Epoch 2/10
22/22 [==============================] - 50s 2s/step - loss: 0.8527 - accuracy: 0.6182 - val_loss: 0.7900 - val_accuracy: 0.6483
Epoch 3/10
22/22 [==============================] - 49s 2s/step - loss: 0.7546 - accuracy: 0.6642 - val_loss: 0.7349 - val_accuracy: 0.6822
Epoch 4/10
22/22 [==============================] - 51s 2s/step - loss: 0.6954 - accuracy: 0.6970 - val_loss: 0.7047 - val_accuracy: 0.6933
Epoch 5/10
22/22 [==============================] - 51s 2s/step - loss: 0.6518 - accuracy: 0.7203 - val_loss: 0.7141 - val_accuracy: 0.6937
Epoch 6/10
22/22 [==============================] - 57s 3s/step - loss: 0.6338 - accuracy: 0.7296 - val_loss: 0.7091 - val_accuracy: 0.6937
Epoch 7/10
22/22 [==============================] - 64s 3s/step - loss: 0.6026 - accuracy: 0.7405 - val_loss: 0.7098 - val_accuracy: 0.6973
Epoch 8/10
22/22 [==

In [8]:
model.save('sentiment.h5')